In [13]:
#引用
from PIL import Image
from Crypto.Cipher import AES
import os

#讀圖片
img=Image.open('penguin.jpg')
img_byte=img.convert("RGB").tobytes()
size=len(img_byte)#範圍
need_to_pad=16-(size%16)#需要填充的數量
s=bytes([255])
for i in range(need_to_pad):#填充
    img_byte+=s
size=len(img_byte)#重新給值
count=int(size/16)

#XOR函式
def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

#加密&解密
IV=os.urandom(16)#IV
XOR_lis=[]#拿來XOR的陣列
XOR_lis.append(IV)
key=os.urandom(16)#16byte金鑰
cipher=AES.new(key,AES.MODE_ECB)
img_c=b''#存加密的
img_d=b''#存解密的

# #加
xor_count=0
for i in range(0,size,16):#範圍內每次加16
    be_encrypt=img_byte[i:i+16]
    be_encrypt=byte_xor(be_encrypt,XOR_lis[xor_count])
    text=cipher.encrypt(be_encrypt)
    XOR_lis.append(text)
    xor_count+=1
    img_c+=text
# #解
decrypt_count=0
for i in range(0,size,16):
    be_decrypt=img_c[i:i+16]
    text_decrypt=cipher.decrypt(be_decrypt)
    text_decrypt=byte_xor(text_decrypt,XOR_lis[decrypt_count])
    decrypt_count+=1
    img_d+=text_decrypt
    
#輸出    
CBC_encrypt=Image.frombytes("RGB",img.size,img_c)
CBC_encrypt.save('./CBC_encrypt.png')
CBC_decrypt=Image.frombytes("RGB",img.size,img_d)
CBC_decrypt.save('./CBC_decrypt.png')